## Data Cleaning

> This file selects relevant time series from the AMECO (2018 Fall edition) and from the JST dataset. Followingly it calculates relevant measured based on the time series. If running for the first time, concacenate the seperate AMECO-tables. Also you may install all relevant packages the analysis of the model. 

In [ ]:
#Settings
install_packages = False
concacenate = True

#Install packages
if install_packages == True:   
    !pip install numpy
    !pip install pandas
    !pip install scipy
    !pip install statistics
    !pip install more-itertools
    !pip install tabulate

#concacenate csv-file
if concacenate == 1:
    file_names = [f'data/AMECO_seperate/AMECO{i}.csv' for i in ["1","2","3","4","5","6","7","8","9","10","11","12","13"]]
    combined_csv = pd.concat([pd.read_csv(f, sep=";") for f in file_names])
    combined_csv.to_csv('data/AMECO.csv', sep=";", index = False)

#Import libraries
import pandas as pd

In [ ]:
AMECO = pd.read_csv("data/AMECO.csv", sep=";")

def get_ameco(code,ISO):
    code = str(code)
    ISO = str(ISO)
    identifier = ISO + "." + code
    return AMECO[AMECO["CODE"] == identifier]

df_jst = pd.read_stata("http://macrohistory.net/JST/JSTdatasetR4.dta", columns =["year", "iso", "eq_dp"])
df_jst['year'] = df_jst['year'].astype(int)

df_pwt = pd.read_stata("https://www.rug.nl/ggdc/docs/pwt91.dta", columns =["year", "countrycode", "labsh"])
df_pwt['year'] = df_pwt['year'].astype(int)

def get_jst(iso):
    return df_jst[(df_jst["iso"]==iso) & df_jst['year'].isin(range(1960,2015+1))].loc[:,"eq_dp"].tolist()

def get_pwt(iso):
    return df_pwt[(df_pwt["countrycode"]==iso) & df_pwt['year'].isin(range(1960,2015+1))].loc[:,"labsh"].tolist()

codes = ['1.0.0.0.NPTD','3.1.0.0.PIGT','3.0.0.0.ZCPIN','1.0.0.0.NLTN','3.0.0.0.ZVGDF','1.0.0.0.ALCD0','1.1.0.0.OIGT','1.0.0.0.OKND','1.0.0.0.AKNDV','1.3.0.0.ISRC','1.0.0.0.NETD','1.0.0.0.NPAN','1.0.0.0.NPON','1.0.0.0.ALCD2','1.0.0.0.UWCD','1.0.0.0.UVGD','1.0.0.0.AVGDK','1.0.0.0.OKND','1.0.0.0.UYGD']
countries = ['BEL','DNK','FIN','FRA','ITA','JPN','NLD','GBR','USA'] #FIN from 1970, JAP from 1981, all others from 1961

In [ ]:
parts = []
for ISO in countries:
    
    #Create dfx with all series for country
    dfx = pd.DataFrame()
    for code in codes:
        data = get_ameco(code,ISO).loc[:,"1960":"2015"]
        data = data.transpose()
        data.columns = [code]
        dfx[code] = data[code]

    dfx["eq_dp"] = get_jst(ISO)
    dfx["labsh"] = get_pwt(ISO)
    dfx["ISO"] = ISO
    dfx["year"] = dfx.index.tolist()
    dfx.reset_index(drop=True)
    #End
    parts.append(dfx)
    
df_data = pd.concat(parts, ignore_index = True)

df_calc = pd.DataFrame()
df_calc["ISO"] = df_data['ISO']
df_calc["year"] = df_data['year']

#Manipulating time series
df_calc['rf'] = df_data['1.3.0.0.ISRC']
df_calc["PD"] = 1/df_data['eq_dp']
df_calc["XK"] = df_data['1.1.0.0.OIGT']/df_data['1.0.0.0.OKND'] *100
df_calc['EmpPop'] = df_data['1.0.0.0.NETD']/df_data['1.0.0.0.NPTD']*100#(df_data['1.0.0.0.NPON']+df_data['1.0.0.0.NPAN']) *100
df_calc['CapShare'] = 100 - (df_data['1.0.0.0.ALCD0']) # ALCD0 for market prices, ALCD2 for factor
df_calc['AvgRet'] = df_calc['CapShare']/df_data['1.0.0.0.AKNDV']


#Growth rates
PopGrowth = [df_data[(df_data['ISO']==ISO)]["1.0.0.0.NPTD"].pct_change()*100 for ISO in countries]
df_calc['PopGrowth'] = pd.concat(PopGrowth, ignore_index = True)

TFPgrowth = [df_data[(df_data['ISO']==ISO)]["3.0.0.0.ZVGDF"].pct_change()*100 for ISO in countries]
df_calc['TFPgrowth'] = pd.concat(TFPgrowth, ignore_index = True)

df_data['temp'] = df_data['3.1.0.0.PIGT']/df_data['3.0.0.0.ZCPIN']

PriceInvt = [df_data[(df_data['ISO']==ISO)]["temp"].pct_change()*100 for ISO in countries]
df_calc['PriceInvt'] = pd.concat(PriceInvt, ignore_index = True)

df_data = df_data.drop(columns="temp")
df_calc = df_calc.set_index('year')
df_calc.to_csv("data/data.csv" ,sep=";", index=True)